In [2]:
%load_ext autoreload
%autoreload 2

In [18]:
import json, os, sys
import numpy as np
from entity_matching import data_utils, binary_resnet, model_utils

import torch.optim as optim
from  torch import nn

In [22]:
sys.path.append('../dice')
from metrics import em_metrics

In [4]:
wd_emb_fpath = 'result/TransE/uniform500_wd/params_37/results/TransE.json'
mbz_emb_fpath = 'result/TransE/uniform500_mbz/params_7/results/TransE.json'
data_dir = '../dice/data/uniform500'

In [5]:
# load datasets and embeddings 
train_loader, val_loader, test_loader = data_utils.get_loaders(data_dir)
mbz_emb = data_utils.load_entity_embeddings(mbz_emb_fpath)
wd_emb = data_utils.load_entity_embeddings(wd_emb_fpath)



In [15]:
# load candidate tails 
candidates = data_utils.load_candidates(data_dir)

# load mbz types (for metrics)
mbz_index2type = data_utils.load_etypes(data_dir)

In [6]:
# initialize different 
model = binary_resnet.BinResNet(mbz_emb, wd_emb)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
loss = nn.CrossEntropyLoss()

In [8]:
model_utils.train(model, optimizer, loss, train_loader, val_loader, candidates, 1, 'cpu', log_interval=1, num_epochs=1)

Train Epoch: 0 	Loss: 0.692926 	Time: 27.94 seconds


In [27]:
x, y_trues, y_preds = model_utils.evaluate_model(model, test_loader, candidates)

  0%|          | 0/154 [00:00<?, ?it/s]


In [28]:
metrics = em_metrics.evaluate_em(x, y_trues, y_preds, mbz_index2type)

In [29]:
metrics

defaultdict(<function metrics.em_metrics.evaluate_em.<locals>.<lambda>>,
            {'total': {'count': 1,
              'fn': 1,
              'fp': 0,
              'precision': 'Na',
              'recall': 0.0,
              'tn': 0,
              'tp': 0},
             'work': defaultdict(int,
                         {'count': 1,
                          'fn': 1,
                          'fp': 0,
                          'precision': 'Na',
                          'recall': 0.0,
                          'tn': 0,
                          'tp': 0})})